**GDELT MongoDB**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymongo
plt.style.use('fivethirtyeight')
from pprint import pprint

**Setup MongoDB**

In [2]:
LOCAL_HOST_SERVER = ''
mongo_client = pymongo.MongoClient(LOCAL_HOST_SERVER)

In [3]:
mongo_client.list_database_names()

['MSBD_2019_2020', 'admin', 'config', 'local']

# GDELT Data Preprocessing

**TODO in Spark with Scala**

1. gdelt_events:
    - Checks on variable types
2. gdelt_mentions:
    - Reduce DataFrame to 2 columns *GLOBALEVENTID* (for merge with gdelt_events) and *mentions* (the rest)
    - Checks on variable types
    - Create column for language of article / mention
3. gdelt_gkg:
    - Transfomring columns with poorly formated lists into dictionnaries
    - Checks on variable types

In [10]:
gdelt_events = pd.read_csv('test_local_gdelt_events.csv', index_col=0)
gdelt_mentions = pd.read_csv('test_local_gdelt_mentions.csv', index_col=0)
gdelt_gkg = pd.read_csv('test_local_gdelt_gkg.csv', index_col=0)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## GDELT Mentions

In [16]:
gdelt_mentions.head()

,GLOBALEVENTID,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras,MentionDocOriginalLanguage
0,410412347,20150218230000,20150218230000,1,dailymaverick.co.za,http://www.dailymaverick.co.za/article/2015-02...,19,-1,4594,4634,1,50,6665,-4.477612,NaN,NaN,eng
1,410412348,20150218230000,20150218230000,1,indiatimes.com,http://timesofindia.indiatimes.com/city/bengal...,2,-1,300,344,1,50,2541,2.078522,NaN,NaN,eng
2,410412349,20150218230000,20150218230000,1,voxy.co.nz,http://www.voxy.co.nz/entertainment/coast-new-...,4,-1,1297,1232,0,10,2576,7.517084,NaN,NaN,eng
3,410412350,20150218230000,20150218230000,1,voxy.co.nz,http://www.voxy.co.nz/entertainment/coast-new-...,4,-1,1298,1233,1,20,2576,7.517084,NaN,NaN,eng
4,410412351,20150218230000,20150218230000,1,eastidahonews.com,http://www.eastidahonews.com/2015/02/neil-patr...,1,-1,103,122,1,100,1432,0.000000,NaN,NaN,eng


In [17]:
gdelt_mentions_processed = gdelt_mentions[['GLOBALEVENTID']]
gdelt_mentions_processed['mentions'] = gdelt_mentions.drop('GLOBALEVENTID', axis=1).to_dict(orient='records')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
gdelt_mentions_processed.head()

,GLOBALEVENTID,mentions
0,410412347,"{'EventTimeDate': 20150218230000, 'MentionTime..."
1,410412348,"{'EventTimeDate': 20150218230000, 'MentionTime..."
2,410412349,"{'EventTimeDate': 20150218230000, 'MentionTime..."
3,410412350,"{'EventTimeDate': 20150218230000, 'MentionTime..."
4,410412351,"{'EventTimeDate': 20150218230000, 'MentionTime..."


In [13]:
gdelt_mentions_processed_bis = gdelt_mentions_processed.groupby('GLOBALEVENTID')['mentions'].apply(list)
gdelt_mentions_processed_bis = gdelt_mentions_processed_bis.reset_index()
gdelt_mentions_processed_bis.head()

,GLOBALEVENTID,mentions
0,403685556,"[{'EventTimeDate': 20150119000000, 'MentionTim..."
1,403686013,"[{'EventTimeDate': 20150119000000, 'MentionTim..."
2,403688003,"[{'EventTimeDate': 20150119000000, 'MentionTim..."
3,403688601,"[{'EventTimeDate': 20150119001500, 'MentionTim..."
4,403692727,"[{'EventTimeDate': 20150119003000, 'MentionTim..."


## GDELT Events

In [14]:
gdelt_events_preprocessed = gdelt_events.merge(gdelt_mentions_processed_bis, on='GLOBALEVENTID', how='left')
print(gdelt_events.shape)
print(gdelt_events_preprocessed.shape)

(29319, 61)
(29319, 62)


In [15]:
gdelt_events_preprocessed[['GLOBALEVENTID', 'SQLDATE', 'ActionGeo_CountryCode', 'mentions']].head()

,GLOBALEVENTID,SQLDATE,ActionGeo_CountryCode,mentions
0,410412347,20140218,SF,"[{'EventTimeDate': 20150218230000, 'MentionTim..."
1,410412348,20140218,IN,"[{'EventTimeDate': 20150218230000, 'MentionTim..."
2,410412349,20140218,AS,"[{'EventTimeDate': 20150218230000, 'MentionTim..."
3,410412350,20140218,NZ,"[{'EventTimeDate': 20150218230000, 'MentionTim..."
4,410412351,20140218,US,"[{'EventTimeDate': 20150218230000, 'MentionTim..."


## GDELT GKG

Columns to format from string to list:

- AllNames
- Amounts
- Counts
- GCAM
- Locations

In [19]:
# pprint(gdelt_gkg.transpose().iloc[:,0].to_dict()[''])

# Insert GDELT